In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

# 下面的代码试图复现malinois preprocessing

In [2]:
df = pd.read_csv('../data/Gosai_MPRA/metadata.csv', comment='#')
df

,Accession,Database,Description,Link,Usage,experiment_accession,file_accession,project,cell_type,ref_accession
0,ENCFF996ECA,ENCODE,MPRA normalized count data for OL27 in SK-N-SH...,https://www.encodeproject.org/files/ENCFF996EC...,MPRA experimental data used for Malinois train...,ENCSR623DJZ,ENCFF304FDH,OL27,SK-N-SH,ENCFF715XTT
1,ENCFF018AMJ,ENCODE,MPRA normalized count data for OL27 in A549 cells,https://www.encodeproject.org/files/ENCFF018AM...,MPRA experimental data used for Malinois train...,ENCSR729IRS,ENCFF439ZER,OL27,A549,ENCFF715XTT
2,ENCFF345ASG,ENCODE,MPRA normalized count data for OL27 in GM12878...,https://www.encodeproject.org/files/ENCFF345AS...,MPRA experimental data used for Malinois train...,ENCSR645CIP,ENCFF725MUJ,OL27,GM12878,ENCFF715XTT
3,ENCFF970OLE,ENCODE,MPRA normalized count data for OL27 in K562 cells,https://www.encodeproject.org/files/ENCFF970OL...,MPRA experimental data used for Malinois train...,ENCSR857JFG,ENCFF172YVF,OL27,K562,ENCFF715XTT
4,ENCFF318XMJ,ENCODE,MPRA normalized count data for OL27 in HepG2 c...,https://www.encodeproject.org/files/ENCFF318XM...,MPRA experimental data used for Malinois train...,ENCSR847SNQ,ENCFF224YJV,OL27,HepG2,ENCFF715XTT
5,ENCFF358MBK,ENCODE,MPRA normalized count data for OL28 in SK-N-SH...,https://www.encodeproject.org/files/ENCFF358MB...,MPRA experimental data used for Malinois train...,ENCSR833GXJ,ENCFF636IWQ,OL28,SK-N-SH,ENCFF384XZU
6,ENCFF379XWL,ENCODE,MPRA normalized count data for OL28 in HepG2 c...,https://www.encodeproject.org/files/ENCFF379XW...,MPRA experimental data used for Malinois train...,ENCSR979TSK,ENCFF847LZR,OL28,HepG2,ENCFF384XZU
7,ENCFF774CHX,ENCODE,MPRA normalized count data for OL28 in K562 cells,https://www.encodeproject.org/files/ENCFF774CH...,MPRA experimental data used for Malinois train...,ENCSR653LWA,ENCFF017YYT,OL28,K562,ENCFF384XZU
8,ENCFF138DJM,ENCODE,MPRA normalized count data for OL28 in A549 cells,https://www.encodeproject.org/files/ENCFF138DJ...,MPRA experimental data used for Malinois train...,ENCSR523VYM,ENCFF282DRV,OL28,A549,ENCFF384XZU
9,ENCFF277DDE,ENCODE,MPRA normalized count data for OL29 in A549 cells,https://www.encodeproject.org/files/ENCFF277DD...,MPRA experimental data used for Malinois train...,ENCSR273YGD,ENCFF025ATT,OL29,A549,ENCFF074MMO


In [3]:
import gzip

def read_fasta_gz_to_dict(fasta_gz_path):
    """
    从 .fasta.gz 文件读取并转换为 {id: sequence} 字典。
    """
    seq_dict = {}
    with gzip.open(fasta_gz_path, "rt") as f:
        seq_id = None
        seq_lines = []
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if seq_id is not None:
                    seq_dict[seq_id] = "".join(seq_lines)
                seq_id = line[1:]  # 去掉 '>'
                seq_lines = []
            else:
                seq_lines.append(line)
        # 处理最后一条序列
        if seq_id is not None:
            seq_dict[seq_id] = "".join(seq_lines)
    return seq_dict


def map_data_project(ol):
    ukbb_list = ['OL27', 'OL28', 'OL29', 'OL30', 'OL31', 'OL32', 'OL33']
    gtex_list = ['OL41', 'OL42', 'OL41_42', 'OL41B']
    ol15_list = ['OL15']
    if ol in ukbb_list:
        return 'UKBB'
    elif ol in gtex_list:
        return 'GTEx'
    elif ol in ol15_list:
        return 'OL15'
    else:
        raise ValueError(f'Unknown OL: {ol}')

In [30]:
def expand_dic(multi_key_dict):
    expanded_dic = {}
    for compound_key, seq in multi_key_dict.items():
        # 去除外层括号（如果存在）
        if compound_key.startswith("(") and compound_key.endswith(")"):
            compound_key = compound_key[1:-1]
        # 用 ; 拆分
        keys = compound_key.split(";")
        for key in keys:
            if key in expanded_dic:
                if expanded_dic[key] != seq:
                    print(f"expanded conflict key {key}")
                    expanded_dic[key] = None
                else: #已存在且值一致，跳过
                    print(f"expanded conflict key {key}, has same values! skip")
                    continue
            else:
                expanded_dic[key] = seq
    return expanded_dic

metadata_df = pd.read_csv('../data/Gosai_MPRA/metadata.csv')
total_dict = {}
ref_accessions = list(set(metadata_df['ref_accession']))
for ref_accession in ref_accessions:
    if ref_accession == 'ENCFF443RYE;ENCFF728XQT': # 已经有了
        continue
    dic = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/{ref_accession}.fasta.gz")
    dic = expand_dic(dic)
    for key in dic:
        if key in total_dict:
            if total_dict[key] != dic[key]:
                print(f"merge conflict key '{key}' has different values!")
                total_dict[key] = None
        else:
            total_dict[key] = dic[key]
seq_dict = total_dict
# 暂时忽略相同ID不同序列的情况



from collections import Counter

metadata_df = pd.read_csv('../data/Gosai_MPRA/metadata.csv')
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']

dfs_dict = {}
for cell_type in cell_types:
    dfs = []
    for i, row in metadata_df.iterrows():
        if row['cell_type'] == cell_type:
            ref_accession = row['ref_accession']
            # if ref_accession == 'ENCFF443RYE;ENCFF728XQT':
            #     dict1 = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/ENCFF443RYE.fasta.gz")
            #     dict2 = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/ENCFF728XQT.fasta.gz")
            #     seq_dict = (dict1 | dict2)
            # else:
            #     seq_dict = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/{ref_accession}.fasta.gz")
            file_accession = row['file_accession']
            print(file_accession)
            df = pd.read_csv(f"../data/Gosai_MPRA/raw/{file_accession}.tsv", sep='\t', low_memory=False)
            df['OL'] = row['project']
            df['data_project'] = map_data_project(row['project'])
            df['seq'] = df['ID'].map(seq_dict)
            dfs.append(df)
    dfs = pd.concat(dfs).reset_index(drop=True)
    dfs_dict[cell_type] = dfs

merge conflict key '19:54676763:C:T:A:wC:Alt_19-54676814-A' has different values!
merge conflict key '19:54676763:C:T:A:wC:Alt_19-54676814-R' has different values!
merge conflict key '19:54676763:C:T:A:wL:Alt_19-54676814-A' has different values!
merge conflict key '19:54676763:C:T:A:wL:Alt_19-54676814-R' has different values!
merge conflict key '19:54676763:C:T:R:wC:Alt_19-54676814-A' has different values!
merge conflict key '19:54676763:C:T:R:wC:Alt_19-54676814-R' has different values!
merge conflict key '19:54676763:C:T:R:wL:Alt_19-54676814-A' has different values!
merge conflict key '19:54676763:C:T:R:wL:Alt_19-54676814-R' has different values!
ENCFF172YVF
ENCFF017YYT
ENCFF889WPF
ENCFF612YRJ
ENCFF842RUM
ENCFF995HVI
ENCFF744OCG
ENCFF141ZOX
ENCFF090LOS
ENCFF224YJV
ENCFF847LZR
ENCFF866FZN
ENCFF848GPT
ENCFF909YEL
ENCFF112XKL
ENCFF983QNK
ENCFF734CRQ
ENCFF539GVZ
ENCFF521ALH
ENCFF304FDH
ENCFF636IWQ
ENCFF329BLH
ENCFF480BKB
ENCFF329RAT
ENCFF885GWB
ENCFF382RYO
ENCFF094EUR
ENCFF521IVN
ENCFF209

In [31]:
# from collections import Counter

# metadata_df = pd.read_csv('../data/Gosai_MPRA/metadata.csv')

# dfs = []
# for i, row in tqdm(metadata_df.iterrows(), total=len(metadata_df)):

#     ref_accession = row['ref_accession']
#     if ref_accession == 'ENCFF443RYE;ENCFF728XQT':
#         dict1 = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/ENCFF443RYE.fasta.gz")
#         dict2 = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/ENCFF728XQT.fasta.gz")
#         seq_dict = (dict1 | dict2)
#     else:
#         seq_dict = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/{ref_accession}.fasta.gz")
    
#     file_accession = row['file_accession']

#     df = pd.read_csv(f"../data/Gosai_MPRA/raw/{file_accession}.tsv", sep='\t', low_memory=False)
#     df['OL'] = row['project']
#     df['data_project'] = map_data_project(row['project'])
#     df['seq'] = df['ID'].map(seq_dict)
    
#     dfs.append(df)



# cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549', 'GM12878']
# dfs_dict = {c: [] for c in cell_types}

# for i, row in tqdm(metadata_df.iterrows(), total=len(metadata_df)):
#     cell_type = row['cell_type']
#     dfs_dict[cell_type].append(dfs[i])

# for c in cell_types:
#     dfs_dict[c] = pd.concat(dfs_dict[c]).reset_index(drop=True)

In [32]:
def filter_nonnatural_oligos(df):
    cond = (df['chr'].notna())
    df_filtered = df[cond].copy()
    return df_filtered


def filter_seq_is_none(df):
    cond = (df['seq'].notna())
    df_filtered = df[cond].copy()
    return df_filtered



def filter_plasmid_and_rna_count(df):
    # DNA count >= 20, RNA_count > 0
    DNA_cond_1 = (df['DNA_mean'] >= 5) & (df['data_project']=='OL15') # OL15 4 replicates
    DNA_cond_2 = (df['DNA_mean'] >= 4) & (df['data_project']!='OL15') # other 5 replicates
    RNA_cond = (df['exp_mean'] > 0)
    cond = (DNA_cond_1 | DNA_cond_2) & RNA_cond
    df_filtered = df[cond].copy()
    return df_filtered



def filter_log2fc_6std(df):
    std = df['log2FoldChange'].std()
    mean = df['log2FoldChange'].mean()
    cond = (df['log2FoldChange'] >= (mean - 6 * std))
    df_filtered = df[cond].copy()
    return df_filtered



def merge_ukbb_gtex_ol15(df):
    # 定义优先级映射
    priority_map = {'UKBB': 3, 'GTEx': 2, 'OL15': 1}
    # 给 dataframe 新增一列 priority 便于后续比较
    df['priority'] = df['data_project'].map(priority_map)
    if df['priority'].isna().any():
        print('存在 priority 为空的行!!')
    
    # 构造聚合字典：log2FoldChange 做 mean，其余列都用 'first'
    # 如果你的表有很多列，只关心部分字段，也可以只在聚合字典中声明关心的列。
    agg_dict = {}
    for col in df.columns:
        if col == 'log2FoldChange':
            agg_dict[col] = 'mean'
        else:
            # 取第一行，如果同一个 (ID, data_project) 下这些列确实没有冲突
            agg_dict[col] = 'first'

    # 一次分组：对 ID + data_project 分组
    df_agg = df.groupby(['seq', 'data_project'], as_index=False).agg(agg_dict)

    # 对聚合完的数据按优先级降序排；同一个 ID 下，UKBB(3) > GTEx(2) > OL15(1)
    df_agg.sort_values(['seq', 'priority'], ascending=[True, False], inplace=True)

    # 同一个 ID 只保留优先级最高的项目那条记录
    df_final = df_agg.drop_duplicates(subset='seq', keep='first').copy()

    # 不需要 priority 列了，可以删除
    df_final.drop(columns='priority', inplace=True)
    
    return df_final

In [33]:
for cell_type in cell_types:
    # print(cell_type)
    # df_i = dfs_dict[cell_type]
    # print(len(df_i))
    # df_i = filter_plasmid_and_rna_count(df_i)
    # print(len(df_i))
    # df_i = merge_ukbb_gtex_ol15(df_i)
    # print(len(df_i))
    # df_i = filter_nonnatural_oligos(df_i)
    # print(len(df_i))
    # df_i = filter_log2fc_6std(df_i)
    # print(len(df_i))
    # dfs_dict[cell_type] = df_i


    print(cell_type)
    df_i = dfs_dict[cell_type]
    print(len(df_i))
    df_i = filter_nonnatural_oligos(df_i)
    print(len(df_i))
    df_i = filter_seq_is_none(df_i)
    print(len(df_i))
    df_i = filter_plasmid_and_rna_count(df_i)
    print(len(df_i))
    df_i = merge_ukbb_gtex_ol15(df_i)
    print(len(df_i))
    df_i = filter_log2fc_6std(df_i)
    print(len(df_i))
    dfs_dict[cell_type] = df_i

    # 760681?

K562
944435
929435
929413
908338
797652
797652
HepG2
945301
930301
930279
909167
797947
797943
SK-N-SH
945301
930301
930279
908125
797007
797002
HCT116
494592
494592
494582
481305
467892
467892
A549
420718
420718
420706
413624
338278
338225


In [42]:
prepared = {
    ct: df.drop_duplicates('seq').set_index('seq').add_suffix(f'_{ct}')
    for ct, df in dfs_dict.items()
}

all_meta = pd.concat(
    [df[['seq', 'ID', 'chr', 'pos', 'ref_allele', 'alt_allele', 'allele', 'OL', 'data_project']]
     for df in dfs_dict.values()],
    ignore_index=True
)
meta = all_meta.drop_duplicates('seq').set_index('seq')

merged_df = pd.concat([meta] + list(prepared.values()), axis=1, join='outer')
merged_df = merged_df.reset_index()
merged_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,ID_K562,SNP_K562,chr_K562,pos_K562,ref_allele_K562,alt_allele_K562,allele_K562,window_K562,strand_K562,project_K562,haplotype_K562,ctrl_exp_K562,DNA_mean_K562,ctrl_mean_K562,exp_mean_K562,log2FoldChange_K562,lfcSE_K562,stat_K562,pvalue_K562,padj_K562,OL_K562,data_project_K562,start_K562,stop_K562,ID_HepG2,SNP_HepG2,chr_HepG2,pos_HepG2,ref_allele_HepG2,alt_allele_HepG2,allele_HepG2,window_HepG2,strand_HepG2,project_HepG2,haplotype_HepG2,ctrl_exp_HepG2,DNA_mean_HepG2,ctrl_mean_HepG2,exp_mean_HepG2,log2FoldChange_HepG2,lfcSE_HepG2,stat_HepG2,pvalue_HepG2,padj_HepG2,OL_HepG2,data_project_HepG2,start_HepG2,stop_HepG2,ID_SK-N-SH,SNP_SK-N-SH,chr_SK-N-SH,pos_SK-N-SH,ref_allele_SK-N-SH,alt_allele_SK-N-SH,allele_SK-N-SH,window_SK-N-SH,strand_SK-N-SH,project_SK-N-SH,haplotype_SK-N-SH,ctrl_exp_SK-N-SH,DNA_mean_SK-N-SH,ctrl_mean_SK-N-SH,exp_mean_SK-N-SH,log2FoldChange_SK-N-SH,lfcSE_SK-N-SH,stat_SK-N-SH,pvalue_SK-N-SH,padj_SK-N-SH,OL_SK-N-SH,data_project_SK-N-SH,start_SK-N-SH,stop_SK-N-SH,ID_HCT116,SNP_HCT116,chr_HCT116,pos_HCT116,ref_allele_HCT116,alt_allele_HCT116,allele_HCT116,window_HCT116,strand_HCT116,project_HCT116,haplotype_HCT116,ctrl_exp_HCT116,DNA_mean_HCT116,ctrl_mean_HCT116,exp_mean_HCT116,log2FoldChange_HCT116,lfcSE_HCT116,stat_HCT116,pvalue_HCT116,padj_HCT116,OL_HCT116,data_project_HCT116,ID_A549,SNP_A549,chr_A549,pos_A549,ref_allele_A549,alt_allele_A549,allele_A549,window_A549,strand_A549,project_A549,haplotype_A549,ctrl_exp_A549,DNA_mean_A549,ctrl_mean_A549,exp_mean_A549,log2FoldChange_A549,lfcSE_A549,stat_A549,pvalue_A549,padj_A549,OL_A549,data_project_A549
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:R:wC,1,161114366.000,A,T,ref,OL41_42,GTEx,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL41_42,467.400,334.316,434.641,0.379,0.252,1.504,0.133,0.264,OL41_42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,395.638,-0.049,0.143,-0.341,0.733,0.799,OL42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,378.529,-0.119,0.370,-0.321,0.748,0.845,OL42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,333.648,-0.291,0.159,-1.834,0.067,0.127,OL42,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:A:wC,1,161114366.000,A,T,alt,OL41_42,GTEx,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL41_42,25.800,18.766,14.227,-0.401,1.720,-0.233,0.816,0.903,OL41_42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,19.904,0.215,1.005,0.214,0.830,0.876,OL42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,3.438,-2.281,2.928,-0.779,0.436,0.594,OL42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,13.619,-0.329,1.190,-0.276,0.782,0.858,OL42,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,7:89640175:A:G:A:wC,7,89640175.000,A,G,alt,OL29,UKBB,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,17.171,0.956,0.527,1.812,0.070,0.107,OL29,UKBB,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,10.882,0.297,1.291,0.230,0.818,0.877,OL29,UKBB,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,5.091,-0.801,2.904,-0.276,0.783,0.879,OL29,UKBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,

In [43]:
merged_df = merged_df.rename(columns={
    'log2FoldChange_K562': 'K562',
    'log2FoldChange_HepG2': 'HepG2',
    'log2FoldChange_SK-N-SH': 'SK-N-SH',
    'log2FoldChange_HCT116': 'HCT116',
    'log2FoldChange_A549': 'A549',
    
})

merged_df = merged_df[['seq', 'ID', 'chr', 'pos', 'ref_allele', 'alt_allele', 'allele', 'OL', 'data_project', 'K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']]
merged_df['chr'] = 'chr' + merged_df['chr'].astype(str)
merged_df['pos'] = pd.to_numeric(merged_df['pos'], errors='coerce').astype('Int64')
merged_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:R:wC,chr1,161114366,A,T,ref,OL41_42,GTEx,0.379,-0.049,-0.119,NaN,-0.291
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:A:wC,chr1,161114366,A,T,alt,OL41_42,GTEx,-0.401,0.215,-2.281,NaN,-0.329
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,7:89640175:A:G:A:wC,chr7,89640175,A,G,alt,OL29,UKBB,0.956,0.297,-0.801,1.360,NaN
3,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:A:wC,chr19,9294737,G,A,alt,OL41_42,GTEx,-0.101,0.468,-0.154,NaN,-0.168
4,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:R:wC,chr19,9294737,G,A,ref,OL41_42,GTEx,0.154,0.352,-0.016,NaN,0.116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799884,TCCTGAGCCCAAGCAATCATGCCTTCTCTGCCTCCCAAAGTGCAGG...,4:69533217:A:C:A:wC,chr4,69533217,A,C,alt,OL31,UKBB,NaN,NaN,NaN,0.209,NaN
799885,TTGCTAGCGCTGAGCCTTTCGCCTAGAGAGTTGTTCCCCTCAGTGA...,3:90124806:T:A:A:wC,chr3,90124806,T,A,alt,OL29,UKBB,NaN,NaN,NaN,-1.878,NaN
799886,TTTATTAATTCAACTCTTCAAAATTATCAGTGTATCACATAATCAT...,14:32916980:C:T:A:wC,chr14,32916980,C,T,alt,OL31,UKBB,NaN,NaN,NaN,-2.076,NaN
799887,TTTCTGGCCACCAGTCCCCCCTGCCTGGCTGGGCAAGCCATGGCTC...,13:29079001:T:A:R:wC,chr13,29079001,T,A,ref,OL31,UKBB,NaN,NaN,NaN,-1.299,NaN


In [ ]:
merged_df = merged_df[merged_df['seq'].str.len() == 200].reset_index(drop=True)
merged_df

In [45]:
merged_df = merged_df[(merged_df[['K562', 'HepG2', 'SK-N-SH']].notna().all(axis=1))].reset_index(drop=True)
merged_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:R:wC,chr1,161114366,A,T,ref,OL41_42,GTEx,0.379,-0.049,-0.119,NaN,-0.291
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:A:wC,chr1,161114366,A,T,alt,OL41_42,GTEx,-0.401,0.215,-2.281,NaN,-0.329
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,7:89640175:A:G:A:wC,chr7,89640175,A,G,alt,OL29,UKBB,0.956,0.297,-0.801,1.360,NaN
3,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:A:wC,chr19,9294737,G,A,alt,OL41_42,GTEx,-0.101,0.468,-0.154,NaN,-0.168
4,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:R:wC,chr19,9294737,G,A,ref,OL41_42,GTEx,0.154,0.352,-0.016,NaN,0.116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCA...,19:51495611:C:T:R:wC,chr19,51495611,C,T,ref,OL41_42,GTEx,2.609,1.610,1.555,NaN,1.931
760675,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCA...,19:51495611:C:T:A:wC,chr19,51495611,C,T,alt,OL41_42,GTEx,2.988,0.760,-0.777,NaN,1.527
760676,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,6:153341374:C:T:R:wC,chr6,153341374,C,T,ref,OL41_42,GTEx,0.291,-0.115,0.004,NaN,-0.402
760677,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACGGAGTCTTG...,2:181602082:C:T:R:wC,chr2,181602082,C,T,ref,OL41_42,GTEx,2.276,2.067,1.982,NaN,2.346


In [46]:
merged_df = merged_df.sort_values(by=['chr', 'pos']).reset_index(drop=True)
merged_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.554,0.416,0.442,NaN,-0.288
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-0.704,0.668,0.539,NaN,0.159
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-0.659,1.152,0.205,NaN,1.051
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.391,1.270,0.752,NaN,1.100
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.007,-0.211,0.499,NaN,0.258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,None,None,None,OL15,OL15,1.381,1.846,1.702,NaN,NaN
760675,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,None,None,None,OL15,OL15,-0.306,-0.107,-0.080,NaN,NaN
760676,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,None,None,None,OL15,OL15,0.363,0.261,0.060,NaN,NaN
760677,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,None,None,None,OL15,OL15,0.395,0.378,-0.030,NaN,NaN


In [47]:
merged_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_760679.tsv', sep='\t', index=False)

In [2]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_760679.tsv', sep='\t')
MPRA_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.554,0.416,0.442,NaN,-0.288
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-0.704,0.668,0.539,NaN,0.159
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-0.659,1.152,0.205,NaN,1.051
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.391,1.270,0.752,NaN,1.100
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.007,-0.211,0.499,NaN,0.258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,1.381,1.846,1.702,NaN,NaN
760675,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.306,-0.107,-0.080,NaN,NaN
760676,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,0.363,0.261,0.060,NaN,NaN
760677,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,0.395,0.378,-0.030,NaN,NaN


In [6]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
MPRA_df[cell_types] = (MPRA_df[cell_types] - MPRA_df[cell_types].mean(axis=0)) / MPRA_df[cell_types].std(axis=0)
MPRA_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,HCT116,A549
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.018,-0.048,0.017,-0.708,NaN
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-1.043,0.191,0.099,-0.262,NaN
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-1.005,0.650,-0.185,0.627,NaN
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.779,0.762,0.281,0.676,NaN
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.455,-0.643,0.066,-0.164,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,0.716,1.309,1.089,NaN,NaN
760675,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.708,-0.544,-0.427,NaN,NaN
760676,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,-0.143,-0.195,-0.308,NaN,NaN
760677,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,-0.116,-0.084,-0.385,NaN,NaN


In [7]:
MPRA_df.describe()

,pos,K562,HepG2,SK-N-SH,HCT116,A549
count,760679.000,760679.000,760679.000,760679.000,448103.000,319496.000
mean,75202958.598,-0.000,-0.000,0.000,0.000,-0.000
std,57521411.054,1.000,1.000,1.000,1.000,1.000
min,14677.000,-5.360,-5.745,-5.762,-5.956,-6.017
25%,28973267.000,-0.564,-0.597,-0.569,-0.567,-0.504
50%,62363640.000,-0.283,-0.256,-0.206,-0.272,-0.203
75%,112600466.000,0.206,0.284,0.285,0.208,0.215
max,249200985.000,7.854,7.705,8.315,7.761,11.771


In [9]:
MPRA_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_760679.tsv', sep='\t', index=False)

In [48]:
original_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_nature.tsv', sep='\t')
original_df

/tmp/ipykernel_122513/3163262775.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_nature.tsv', sep='\t')


,ID,chr,data_project,OL,K562,HepG2,SK-N-SH,seq
0,7:70038969:G:T:A:wC,chr7,UKBB,29,0.061,0.234,0.047,CCTGGTCTTTCTTGCTAAATAAACATATCGTGCATCATCCAGATCT...
1,1:192696196:C:T:A:wC,chr1,UKBB,33,0.380,0.005,-0.244,CATAAAGATGAGGCTTGGCAAAGAACATCTCTCGGTGCCTCCCATT...
2,1:211209457:C:T:A:wC,chr1,UKBB,33,0.037,0.385,-0.005,CATAAAGCCAATCACTGAGATGACAAGTACTGCCAGGAAAGAAGGC...
3,15:89574440:GT:G:A:wC,chr15,UKBB,33,4.465,4.107,2.870,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...
4,15:89574440:GT:G:R:wC,chr15,UKBB,33,4.509,4.116,3.040,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...
...,...,...,...,...,...,...,...,...
798059,4:44680358:NA:NA,chr4,CRE,15,7.444,5.344,6.585,CAGTAGTAAGAAAGAGACAATGCAAAGGAATTGGCACAGCACTCAG...
798060,18:9125893:NA:NA,chr18,CRE,15,-0.205,-0.157,-0.209,CAGTACTGCTGGCCCCAGAAAAGCCCCTCTCCTTATACCCTAGGCC...
798061,12:33905808:NA:NA,chr12,CRE,15,1.218,0.614,0.570,CAGTACCTTGTCCCCACTTCCCATTTGGCCTCTGGCAGAGGAGGAG...
798062,3:128145854:NA:NA,chr3,CRE,15,-0.222,-0.339,-0.818,CAGTACACCCCAGCTTCCAAAGGCCTTCTGTGACAAAGAGAGACTA...


In [51]:
original_df[original_df['seq'].duplicated(keep=False)]

,ID,chr,data_project,OL,K562,HepG2,SK-N-SH,seq


In [52]:
original_df[original_df['ID'].duplicated(keep=False)]

,ID,chr,data_project,OL,K562,HepG2,SK-N-SH,seq
54562,19:54676763:C:T:R:wL:Alt_19-54676814-A,chr19,UKBB,32,0.019,0.702,-1.027,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...
54564,19:54676763:C:T:A:wL:Alt_19-54676814-A,chr19,UKBB,32,0.007,0.518,-1.589,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...
64731,19:54676763:C:T:A:wC:Alt_19-54676814-A,chr19,UKBB,32,0.118,0.572,0.028,CCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATCT...
64749,19:54676763:C:T:R:wC:Alt_19-54676814-A,chr19,UKBB,32,0.086,0.553,-0.242,CCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATCT...
651490,19:54676763:C:T:R:wC:Alt_19-54676814-A,chr19,GTEX,42,-0.386,0.630,-0.031,TCCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATC...
651491,19:54676763:C:T:A:wC:Alt_19-54676814-A,chr19,GTEX,42,-0.235,0.798,0.220,TCCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATC...
696845,19:54676763:C:T:A:wL:Alt_19-54676814-A,chr19,GTEX,42,0.371,1.033,-0.001,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...
696852,19:54676763:C:T:R:wL:Alt_19-54676814-A,chr19,GTEX,42,0.097,0.995,-1.084,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...


In [54]:
merged_df = pd.merge(original_df, MPRA_df, on='seq', how='outer')
merged_df

,ID_x,chr_x,data_project_x,OL_x,K562_x,HepG2_x,SK-N-SH_x,seq,ID_y,chr_y,pos,ref_allele,alt_allele,allele,OL_y,data_project_y,K562_y,HepG2_y,SK-N-SH_y,A549,HCT116
0,1:161114366:A:T:R:wC,chr1,GTEX,42,0.290,-0.062,0.013,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:R:wC,chr1,161114366.000,A,T,ref,OL41_42,GTEx,0.379,-0.049,-0.119,NaN,-0.291
1,1:161114366:A:T:A:wC,chr1,GTEX,42,0.415,-0.020,-1.257,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:A:wC,chr1,161114366.000,A,T,alt,OL41_42,GTEx,-0.401,0.215,-2.281,NaN,-0.329
2,7:89640175:A:G:A:wC,chr7,UKBB,29,0.956,0.293,-0.780,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,7:89640175:A:G:A:wC,chr7,89640175.000,A,G,alt,OL29,UKBB,0.956,0.297,-0.801,1.360,NaN
3,19:9294737:G:A:A:wC,chr19,GTEX,42,-0.078,0.456,-0.075,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:A:wC,chr19,9294737.000,G,A,alt,OL41_42,GTEx,-0.101,0.468,-0.154,NaN,-0.168
4,19:9294737:G:A:R:wC,chr19,GTEX,42,0.135,0.397,0.128,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:R:wC,chr19,9294737.000,G,A,ref,OL41_42,GTEx,0.154,0.352,-0.016,NaN,0.116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798111,19:51495611:C:T:A:wC,chr19,GTEX,42,3.992,2.314,1.359,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCA...,19:51495611:C:T:A:wC,chr19,51495611.000,C,T,alt,OL41_42,GTEx,2.988,0.760,-0.777,NaN,1.527
798112,6:153341374:C:T:R:wC,chr6,GTEX,41,0.341,0.254,-0.055,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,6:153341374:C:T:R:wC,chr6,153341374.000,C,T,ref,OL41_42,GTEx,0.291,-0.115,0.004,NaN,-0.402
798113,6:153341374:C:T:A:wC,chr6,GTEX,41,-0.156,0.364,-0.092,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798114,2:181602082:C:T:R:wC,chr2,GTEX,42,2.206,2.024,1.885,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACGGAGTCTTG...,2:181602082:C:T:R:wC,chr2,181602082.000,C,T,ref,OL41_42,GTEx,2.276,2.067,1.982,NaN,2.346


In [55]:
for cell_type in ['HepG2', 'K562', 'SK-N-SH']:
    x = merged_df[f'{cell_type}_x']
    y = merged_df[f'{cell_type}_y']
    print(pearson(x, y))

(0.9988171116094543, 0.0)
(0.9987607154598933, 0.0)
(0.9987628073694255, 0.0)
